In [96]:
import pandas as pd
from clean_crash_data import crash_data

# Where are crashes happening most frequently?

In [97]:
crash_data[['LOCALREPORTNO', 'ADDRESS_X']].groupby('ADDRESS_X').nunique()

,LOCALREPORTNO
ADDRESS_X,
% N I75 AT 2.7%,1
0-X W 562 AT I75,4
0-X W NORWOOD LATERAL,9
0.0X S I71 EX,1
0.X E NORWOOD LATERAL,7
...,...
XX WILLIAM HOWARD,2
XX WILLIAM HOWARD TAFT,19
XX WILLIAM HOWARD TAFT RD,156


It looks like there are very few locations where there are more than 100 traffic incidents.

In [98]:
crash_data.ADDRESS_X.nunique()

13347

In [99]:
# crash_data[['LOCALREPORTNO', 'ADDRESS_X']].groupby('ADDRESS_X').nunique().plot(kind='bar')

There are 13,573 unique addresses out of 328,558 records / 169,706 reports.

In [100]:
addr_incs = crash_data[['ADDRESS_X', 'LOCALREPORTNO']]

In [101]:
addr_incs_grp = addr_incs.groupby('ADDRESS_X').nunique().sort_values(by='LOCALREPORTNO', ascending=False)

In [102]:
# addr_incs_grp.plot(kind='bar')

In [103]:
addr_incs_grp.describe()

,LOCALREPORTNO
count,13347.000000
mean,12.713793
std,29.533590
min,1.000000
25%,1.000000
50%,3.000000
75%,10.000000
max,575.000000


While most addresses have 10 or fewer crashes over the last decade, the most crashes that have occurred at a single address is 575! Let's look at the top 25%.

In [104]:
top_qtr = addr_incs_grp.loc[addr_incs_grp.LOCALREPORTNO > 10]

In [105]:
top_qtr_data = crash_data[crash_data.ADDRESS_X.isin(top_qtr.index)]

In [106]:
top_qtr_data.LOCALREPORTNO.nunique()

141371

In [107]:
top_qtr_crashes = top_qtr_data.groupby('ADDRESS_X').nunique().sort_values(by='LOCALREPORTNO', ascending=False)['LOCALREPORTNO']

In [108]:
top_qtr_crashes[0:10]

ADDRESS_X
2XX W MITCHELL AVE    575
1XX W MITCHELL AVE    483
61XX GLENWAY AVE      472
2XX CALHOUN ST        404
44XX COLERAIN AVE     399
49XX GLENWAY AVE      375
16XX WESTWOOD AVE     368
35XX READING RD       352
39XX VINE ST          350
55XX COLERAIN AVE     349
Name: LOCALREPORTNO, dtype: int64

2XX W Mitchell Avenue is the location with 575 crashes. Interestingly, 1XX W Mitchell Avenue is the next highest at 483 crashes.

In [109]:
len(top_qtr_crashes)

3204

3,204 addresses make up the top quarter of crashes. Streets would be a more useful measure (but there might be different clusters on the same street)

# Crashes by Street

In [111]:
data_by_st = crash_data
data_by_st['STREET'] = data_by_st.ADDRESS_X.str.replace('^\d+X+ ', '', regex=True)

In [113]:
data_by_st.STREET

0         S I75 AT 2-7 MM
1            E LIBERTY ST
2            E LIBERTY ST
3             CENTRAL AVE
4             CENTRAL AVE
               ...       
328553        SYCAMORE ST
328554          WALNUT ST
328555          WALNUT ST
328556       HARRISON AVE
328557       HARRISON AVE
Name: STREET, Length: 328558, dtype: object

In [117]:
data_by_st.STREET.nunique()

6446

There are about 6,446 unique streets with car crashes.

In [122]:
data_by_st.describe()

,LATITUDE_X,LONGITUDE_X,AGE,CRASHSEVERITY,INJURIES,MANNEROFCRASH,ROADCONTOUR,ROADSURFACE,ROADCLASS,LIGHTCONDITIONS,ROADCONDITIONS
count,3.285290e+05,3.285270e+05,287961.000000,328551.0,328250.0,328537.0,328537.0,328537.0,203133.000000,328537.0,328537.0
mean,2.633354e+03,-1.199095e+04,37.465070,3.481904,2.663829,4.262655,1.469451,1.853091,2.825745,1.79539,1.551956
std,1.469781e+06,6.819152e+06,16.585971,1.080332,1.837182,2.441715,0.857321,0.408198,1.484126,1.336644,4.998746
min,-8.449023e+01,-3.908552e+09,0.000000,1.0,1.0,1.0,1.0,1.0,0.000000,1.0,1.0
25%,3.911740e+01,-8.454953e+01,24.000000,3.0,1.0,2.0,1.0,2.0,2.000000,1.0,1.0
50%,3.913663e+01,-8.451502e+01,34.000000,3.0,1.0,6.0,1.0,2.0,2.000000,1.0,1.0
75%,3.916096e+01,-8.448473e+01,49.000000,5.0,5.0,6.0,2.0,2.0,4.000000,3.0,1.0
max,8.424329e+08,1.000000e-03,933.000000,5.0,5.0,9.0,9.0,9.0,10.000000,9.0,99.0


In [119]:
data_by_st.groupby(by='STREET').nunique().sort_values(by='LOCALREPORTNO', ascending=False).LOCALREPORTNO

STREET
READING RD           6997
GLENWAY AVE          5378
VINE ST              4698
HARRISON AVE         2988
COLERAIN AVE         2639
                     ... 
OAKTREE                 1
BRACKETT AL             1
OBERLIN BOULEVARD       1
OBRYON                  1
  VINE ST               1
Name: LOCALREPORTNO, Length: 6446, dtype: int64

When grouped by street rather than address, Reading Rd has the most crashes, with Glenway second.

# W Mitchell Avea

In [82]:
w_mitchell_avenue = crash_data[crash_data['ADDRESS_X'].str.contains('W MITCHELL AV', case=False, na=False)]

In [83]:
w_mitchell_avenue.groupby(by='ADDRESS_X').nunique().sort_values(by='LOCALREPORTNO', ascending=False).LOCALREPORTNO

ADDRESS_X
2XX W MITCHELL AVE     575
1XX W MITCHELL AVE     483
XX W MITCHELL AVE      260
X W MITCHELL AVE       101
W MITCHELL AVE          63
45XX W MITCHELL AVE     54
44XX W MITCHELL AVE     52
3XX W MITCHELL AVE      24
46XX W MITCHELL AVE      8
2XX  W MITCHELL AVE      1
47XX W MITCHELL AVE      1
4XX W MITCHELL AVE       1
Name: LOCALREPORTNO, dtype: int64

Google Maps From 4600 W Mitchell Ave to 1 W Mitchell Ave
https://www.google.com/maps/dir/1+W+Mitchell+Ave,+Cincinnati,+OH+45217/4600+W+Mitchell+Ave,+Cincinnati,+OH+45232/@39.1661648,-84.5191883,15z/data=!3m1!4b1!4m13!4m12!1m5!1m1!1s0x8841b36d88f31241:0x8e4b4ba441fad80e!2m2!1d-84.5044751!2d39.1603458!1m5!1m1!1s0x8841b35d97dc6419:0xb01843254e6d7e95!2m2!1d-84.5149961!2d39.1719619

In [84]:
crashes_on_w_mitchell = w_mitchell_avenue.LOCALREPORTNO.nunique()

In [85]:
crashes_on_w_mitchell

1623

In [86]:
crashes_on_w_mitchell / crash_data.LOCALREPORTNO.nunique()

0.009563598222808858

W Mitchell Avenue accounts for .96% of crashes over the last decade, so that doesn't sound extreme.

# Glenway ave

In [87]:
glenway_ave = crash_data[crash_data['ADDRESS_X'].str.contains('GLENWAY', case=False, na=False)]

In [92]:
crashes_on_glenway = glenway_ave.LOCALREPORTNO.nunique()

In [93]:
crashes_on_glenway

6018

There are actually more crashes that occur on Glenway Avenue than W Mitchell Ave.

In [88]:
glenway_ave.groupby(by='ADDRESS_X').nunique().sort_values(by='LOCALREPORTNO', ascending=False).LOCALREPORTNO

ADDRESS_X
61XX GLENWAY AVE    472
49XX GLENWAY AVE    375
53XX GLENWAY AVE    251
60XX GLENWAY AVE    237
48XX GLENWAY AVE    225
                   ... 
64XX GLENWAY AVE      2
65XX GLENWAY AVE      2
GLENWAY AVE           2
43XX GLENWAY          1
69XX GLENWAY AVE      1
Name: LOCALREPORTNO, Length: 85, dtype: int64

In [94]:
crashes_on_glenway / crash_data.LOCALREPORTNO.nunique()

0.03546132723651491

Crashes on Glenway Avenue account for 3.55% of all crashes.

In [ ]:
top_qtr_crashes.filter(regex='GLENWAY').sort_index()

ADDRESS_X
23XX GLENWAY AVE     88
24XX GLENWAY AVE     52
25XX GLENWAY AVE     28
26XX GLENWAY AVE     22
27XX GLENWAY AVE     17
                   ... 
61XX GLENWAY         54
61XX GLENWAY AVE    472
62XX GLENWAY         12
62XX GLENWAY AVE    151
63XX GLENWAY AVE     30
Name: LOCALREPORTNO, Length: 66, dtype: int64

Glenway Avenue-- it's a pretty big strip, maybe separating into clusters would be more useful. https://www.google.com/maps/dir/6300+Glenway+Ave,+Cincinnati,+OH+45211/2300+Glenway+Ave,+Cincinnati,+OH+45204/@39.1250156,-84.6122592,13z/data=!4m13!4m12!1m5!1m1!1s0x8841ca6665f3b8a9:0xb7a4b0818c4ab315!2m2!1d-84.6227301!2d39.1472094!1m5!1m1!1s0x8841b6834b810fab:0x2e34cd800837f19b!2m2!1d-84.5537986!2d39.105488

In [ ]:
top_qtr_crashes

ADDRESS_X
2XX W MITCHELL AVE    575
1XX W MITCHELL AVE    483
61XX GLENWAY AVE      472
2XX CALHOUN ST        404
44XX COLERAIN AVE     399
                     ... 
51XX GRAFTON AVE       11
32XX DAYTONA AVE       11
51XX RIDGE AVE         11
24XX BEEKMAN           11
16XX HEWITT AVE        11
Name: LOCALREPORTNO, Length: 3204, dtype: int64

# Reading Road

In [124]:
crashes_on_reading = crash_data[crash_data.ADDRESS_X.str.contains('READING', na=False)]

In [127]:
num_crashes_on_reading = crashes_on_reading.groupby('ADDRESS_X').nunique().sort_values(by='LOCALREPORTNO', ascending=False).LOCALREPORTNO

In [135]:
num_crashes_on_reading.sort_index()[120:]

ADDRESS_X
71XX READING RD              295
71XX READING RD.               1
72XX READING                   3
72XX READING RD              100
73XX READING                   1
73XX READING RD               70
74XX READING RD               14
75XX READING                   7
75XX READING RD              154
76XX  READING RD               2
76XX READING                  18
76XX READING RD              193
76XX READING RD.               1
77XX READING                  15
77XX READING RD              180
78XX READING RD               23
79XX READING                   3
79XX READING RD               45
7XX READING                    6
7XX READING RD               140
7XX READING ROAD               1
82XX READING RD                1
9XX READING RD                 1
N I471 AT READING              1
N I71 AT READING               1
READING RD                     3
W 562 WO READING              10
WB SR 562 RAMP TO READING      4
Name: LOCALREPORTNO, dtype: int64